<a href="https://colab.research.google.com/github/eriksali/DNN_2023_DL/blob/main/running_Input_dependent_certification_random_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar xzf imagenette2.tgz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1485M  100 1485M    0     0  36.2M      0  0:00:40  0:00:40 --:--:-- 34.9M


In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from tqdm import tqdm

# Define the model architecture
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(128 * 16 * 16, 256)
        self.fc2 = torch.nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the random smoothing function
def random_smoothing(model, images, n_samples=10, epsilon=0.03):
    with torch.no_grad():
        model.eval()
        smooth_probs = torch.zeros((len(images), 10))
        for i in range(n_samples):
            smooth_probs += F.softmax(model(images + torch.randn_like(images) * epsilon), dim=1)
        smooth_probs /= n_samples
    return smooth_probs

# Define the evaluation function
def evaluate(model, dataloader, n_samples=10, epsilon=0.03):
    model.eval()
    total_correct = 0
    total_samples = 0
    total_certified = 0
    for images, labels in tqdm(dataloader):
        images = images.cuda()
        labels = labels.cuda()
        # Calculate the prediction of the model
        preds = model(images)
        # Calculate the accuracy of the model
        total_correct += (preds.argmax(dim=1) == labels).sum().item()
        total_samples += len(images)
        # Calculate the certified accuracy of the model using random smoothing
        smooth_probs = random_smoothing(model, images, n_samples, epsilon)
        smoothed_preds = smooth_probs.argmax(dim=1)
        certified_correct = ((smoothed_preds == preds.argmax(dim=1)) & (smooth_probs.max(dim=1).values >= 0.9)).sum().item()
        total_certified += certified_correct
    # Calculate and return the accuracies
    accuracy = 100.0 * total_correct / total_samples
    certified_accuracy = 100.0 * total_certified / total_samples
    return accuracy, certified_accuracy

# Define the data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training and test data
train_data = ImageFolder('imagenette2/train', transform=transform)
test_data = ImageFolder('imagenette2/val', transform=transform)

batch_size = 32
num_workers = 8
num_classes = 10
learning_rate = 0.001

# Define the dataloaders for training and test data
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

# Define the model
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

import torch.optim as optim
# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Define the smoothing parameters
epsilon = 0.1
num_smooth = 50
num_epochs = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Train the model
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()
    
    # Train the model on the training data
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Perform random smoothing
        smoothed_images = torch.zeros_like(images)
        for j in range(num_smooth):
            smoothed_images += torch.randn_like(images) * epsilon
        smoothed_images = smoothed_images.clamp(0, 1)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass and compute the loss
        outputs = model(smoothed_images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize the weights
        loss.backward()
        optimizer.step()
        
    # Set the model to eval mode
    model.eval()
    
    # Evaluate the model on the test data
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Perform random smoothing
        smoothed_images = torch.zeros_like(images)
        for j in range(num_smooth):
            smoothed_images += torch.randn_like(images) * epsilon
        smoothed_images = smoothed_images.clamp(0, 1)
        
        # Forward pass and compute the predictions
        outputs = model(smoothed_images)
        _, predicted = torch.max(outputs.data, 1)
        
        # Compute the accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    accuracy = 100 * correct / total
    print('Epoch [{}/{}], Test Accuracy: {} %'.format(epoch+1, num_epochs, accuracy))


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import numpy as np

# Load the Imagenette2 dataset
train_dataset = datasets.ImageFolder(
    root='./imagenette2/train',
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
)

test_dataset = datasets.ImageFolder(
    root='./imagenette2/val',
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
)

# Define the model architecture
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model and define the loss function and optimizer
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model for 10 epochs
num_epochs = 2
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Epoch [{}/{}], Test Accuracy: {} %'.format(epoch+1, num_epochs, test_acc))


ValueError: ignored

In [32]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import time

# Define the dataset transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_loss = 0.0
correct = 0
total = 0
num_classes = 10
# Load the model
model = torchvision.models.resnet18(pretrained=True)
##model.eval()
'''
# Define the evaluation function with input-dependent certification
def eval(model, device, test_loader, epsilon, criterion, batch_size=128):
    model.eval()

    smoothing = int(np.ceil(epsilon * 255))
    for data, target in test_loader(batch_size=128):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            # Generate a random smoothing matrix
            if smoothing > 0:
                smooth = torch.zeros_like(data).float()
                smooth.normal_(0, smoothing)
                data_smooth = torch.clamp(data + smooth, 0, 1)
            else:
                data_smooth = data
            # Predict the class labels
            output = model(data_smooth)
            # Compute the loss
            test_loss += criterion(output, target).item() * data.size(0)
            # Compute the accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    # Compute the certification accuracy
    cert_acc = 0.0
    for c in range(num_classes):
        for data, target in test_loader(batch_size=128):
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                # Generate a random smoothing matrix
                if smoothing > 0:
                    smooth = torch.zeros_like(data).float()
                    smooth.normal_(0, smoothing)
                    data_smooth = torch.clamp(data + smooth, 0, 1)
                else:
                    data_smooth = data
                # Predict the class labels
                output = model(data_smooth)
                # Compute the robustness margin
                margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=1)[0]

                # Compute the certification accuracy
                correct_c = (margin >= -epsilon).sum().item()
                cert_acc += correct_c
    cert_acc /= num_classes * len(test_loader.dataset)
    # Print the results
    print('Test Loss: {:.6f}, Accuracy: {:.4f}%, Cert Accuracy: {:.4f}%'.format(
        test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc))
    return test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc
'''


def eval(model, device, test_loader, epsilon, criterion, batch_size=128):
    model.eval()

    smoothing = int(np.ceil(epsilon * 255))
    for data, target in test_loader(batch_size=128):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            # Generate a random smoothing matrix
            if smoothing > 0:
                smooth = torch.zeros_like(data).float()
                smooth.normal_(0, smoothing)
                data_smooth = torch.clamp(data + smooth, 0, 1)
            else:
                data_smooth = data
            # Predict the class labels
            output = model(data_smooth)
            # Reshape the output tensor
            output = output.view(-1, num_classes)
            # Compute the loss
            test_loss += criterion(output, target).item() * data.size(0)
            # Compute the accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    # Compute the certification accuracy
    cert_acc = 0.0
    for c in range(num_classes):
        for data, target in test_loader(batch_size=128):
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                # Generate a random smoothing matrix
                if smoothing > 0:
                    smooth = torch.zeros_like(data).float()
                    smooth.normal_(0, smoothing)
                    data_smooth = torch.clamp(data + smooth, 0, 1)
                else:
                    data_smooth = data
                # Predict the class labels
                output = model(data_smooth)
                # Reshape the output tensor
                output = output.view(-1, num_classes)
                # Compute the robustness margin
                margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=1)[0]

                # Compute the certification accuracy
                correct_c = (margin >= -epsilon).sum().item()
                cert_acc += correct_c
    cert_acc /= num_classes * len(test_loader.dataset)
    # Print the results
    print('Test Loss: {:.6f}, Accuracy: {:.4f}%, Cert Accuracy: {:.4f}%'.format(
        test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc))
    return test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc



# Set the random seed for reproducibility
torch.manual_seed(1234)

# Define the device and hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define the device and hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
momentum = 0.9
weight_decay = 0.0005
epsilon = 0.05
num_iterations = 2


batch_size = 128 # or 128 if you retrain the model with a smaller batch size

# Load the dataset
trainset = torchvision.datasets.ImageFolder(root='imagenette2/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testset = torchvision.datasets.ImageFolder(root='imagenette2/val', transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

radius = 0.1
# Randomly sample Gaussian noise as the smoothing noise
def sample_noise(image):
    noise = torch.randn_like(image) * radius
    return noise


def randomized_score(images, labels):
    total_score = torch.zeros(len(images), num_classes*num_iterations).to(device)

    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        if num_iterations == 1:
            total_score[:, :] += F.softmax(output, dim=1).repeat(1, num_classes)
        else:
            total_score[:, i*num_classes:(i+1)*num_classes] = F.softmax(output, dim=1)
    
    total_score = torch.cat([total_score[:, i*num_classes:(i+1)*num_classes] for i in range(num_iterations)], dim=1)
    return total_score / num_iterations



total_loss = 0.0
total_acc = 0.0
total_cert_acc = 0.0
total_cert_radius = 0.0
total_cert_rate = 0.0
total_cert_correct = 0.0

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Compute the regular loss
        output = model(images)
        loss = criterion(output, labels)
        total_loss += loss.item() * len(images)
        _, predicted = output.max(1)
        total_acc += predicted.eq(labels).sum().item()

        # Compute the randomized smoothing loss
        randomized_scores = randomized_score(images, labels)
        randomized_predicted = randomized_scores.max(1)[1]
        cert_indices = randomized_predicted.eq(predicted)
        cert_images = images[cert_indices]
        cert_labels = labels[cert_indices]
        cert_scores = randomized_scores[cert_indices]
        cert_radius = torch.norm(sample_noise(cert_images).view(len(cert_images), -1), dim=1).mean()
        cert_correct = cert_scores.max(1)[1].eq(cert_labels).sum().item()
        cert_acc = cert_correct / len(cert_images) if len(cert_images) > 0 else 0.0
        cert_rate = len(cert_images) / len(images)

        total_cert_acc += cert_acc
        total_cert_radius += cert_radius
        total_cert_rate += cert_rate
        total_cert_correct += cert_correct

num_images = len(test_loader.dataset)
avg_loss = total_loss / num_images
avg_acc = total_acc / num_images
avg_cert_acc = total_cert_acc / num_images
avg_cert_radius = total_cert_radius / num_images
avg_cert_rate = total_cert_rate / num_images
avg_cert_correct = total_cert_correct / num_images

print('Regular test accuracy: %.2f%%' % (avg_acc * 100))
print('Certified test accuracy: %.2f%%' % (avg_cert_acc * 100))
print('Average certified radius: %.4f' % avg_cert_radius)
print('Certification rate: %.2f%%' % (avg_cert_rate * 100))
print('Average number of correct labels within certified radius: %.2f' % avg_cert_correct)


# Evaluate the model with input-dependent certification
eval(model, device, test_loader, criterion, num_classes=10, radius=0.1, num_iterations=1, batch_size=128)



RuntimeError: ignored

In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import time

# Define the dataset transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Load the model
model = torchvision.models.resnet18(pretrained=True)
##model.eval()

# Define the evaluation function with input-dependent certification
def eval(model, device, test_loader, epsilon, criterion, batch_size=128):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    num_classes = 10
    smoothing = int(np.ceil(epsilon * 255))
    for data, target in test_loader(batch_size=128):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            # Generate a random smoothing matrix
            if smoothing > 0:
                smooth = torch.zeros_like(data).float()
                smooth.normal_(0, smoothing)
                data_smooth = torch.clamp(data + smooth, 0, 1)
            else:
                data_smooth = data
            # Predict the class labels
            output = model(data_smooth)
            # Compute the loss
            test_loss += criterion(output, target).item() * data.size(0)
            # Compute the accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    # Compute the certification accuracy
    cert_acc = 0.0
    for c in range(num_classes):
        for data, target in test_loader(batch_size=128):
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                # Generate a random smoothing matrix
                if smoothing > 0:
                    smooth = torch.zeros_like(data).float()
                    smooth.normal_(0, smoothing)
                    data_smooth = torch.clamp(data + smooth, 0, 1)
                else:
                    data_smooth = data
                # Predict the class labels
                output = model(data_smooth)
                # Compute the robustness margin
                ##margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=1)[0]
                margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=2)[0]

                # Compute the certification accuracy
                correct_c = (margin >= -epsilon).sum().item()
                cert_acc += correct_c
    cert_acc /= num_classes * len(test_loader.dataset)
    # Print the results
    print('Test Loss: {:.6f}, Accuracy: {:.4f}%, Cert Accuracy: {:.4f}%'.format(
        test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc))
    return test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc

# Set the random seed for reproducibility
torch.manual_seed(1234)

# Define the device and hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
momentum = 0.9
weight_decay = 0.0005
epsilon = 0.05
num_iterations = 1


batch_size = 128 # or 128 if you retrain the model with a smaller batch size

# Load the dataset
trainset = torchvision.datasets.ImageFolder(root='imagenette2/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testset = torchvision.datasets.ImageFolder(root='imagenette2/val', transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

radius = 0.1
# Randomly sample Gaussian noise as the smoothing noise
def sample_noise(image):
    noise = torch.randn_like(image) * radius
    return noise

num_classes = 1000
# Compute the average score of the randomized model over multiple iterations
'''def randomized_score(images, labels):
    total_score = torch.zeros(len(images), num_classes).to(device)
    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        total_score += F.softmax(output, dim=1)
    return total_score / num_iterations'''

def randomized_score(images, labels):
    total_score = torch.zeros(len(images), num_classes*num_iterations).to(device)
    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        total_score[:, i*num_classes:(i+1)*num_classes] += F.softmax(output, dim=1)
    return total_score / num_iterations


total_loss = 0.0
total_acc = 0.0
total_cert_acc = 0.0
total_cert_radius = 0.0
total_cert_rate = 0.0
total_cert_correct = 0.0

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Compute the regular loss
        output = model(images)
        loss = criterion(output, labels)
        total_loss += loss.item() * len(images)
        _, predicted = output.max(1)
        total_acc += predicted.eq(labels).sum().item()

        # Compute the randomized smoothing loss
        randomized_scores = randomized_score(images, labels)
        randomized_predicted = randomized_scores.max(1)[1]
        cert_indices = randomized_predicted.eq(predicted)
        cert_images = images[cert_indices]
        cert_labels = labels[cert_indices]
        cert_scores = randomized_scores[cert_indices]
        cert_radius = torch.norm(sample_noise(cert_images).view(len(cert_images), -1), dim=1).mean()
        cert_correct = cert_scores.max(1)[1].eq(cert_labels).sum().item()
        cert_acc = cert_correct / len(cert_images) if len(cert_images) > 0 else 0.0
        cert_rate = len(cert_images) / len(images)

        total_cert_acc += cert_acc
        total_cert_radius += cert_radius
        total_cert_rate += cert_rate
        total_cert_correct += cert_correct

num_images = len(test_loader.dataset)
avg_loss = total_loss / num_images
avg_acc = total_acc / num_images
avg_cert_acc = total_cert_acc / num_images
avg_cert_radius = total_cert_radius / num_images
avg_cert_rate = total_cert_rate / num_images
avg_cert_correct = total_cert_correct / num_images

print('Regular test accuracy: %.2f%%' % (avg_acc * 100))
print('Certified test accuracy: %.2f%%' % (avg_cert_acc * 100))
print('Average certified radius: %.4f' % avg_cert_radius)
print('Certification rate: %.2f%%' % (avg_cert_rate * 100))
print('Average number of correct labels within certified radius: %.2f' % avg_cert_correct)


# Evaluate the model with input-dependent certification
eval(model, device, test_loader, criterion, num_classes=10, radius=0.1, num_iterations=1, batch_size=128)



KeyboardInterrupt: ignored

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import time

# Define the dataset transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the model
model = torchvision.models.resnet18(pretrained=True)
##model.eval()

# Define the evaluation function with input-dependent certification
def eval(model, device, test_loader, epsilon, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    num_classes = 10
    smoothing = int(np.ceil(epsilon * 255))
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            # Generate a random smoothing matrix
            if smoothing > 0:
                smooth = torch.zeros_like(data).float()
                smooth.normal_(0, smoothing)
                data_smooth = torch.clamp(data + smooth, 0, 1)
            else:
                data_smooth = data
            # Predict the class labels
            output = model(data_smooth)
            # Compute the loss
            test_loss += criterion(output, target).item() * data.size(0)
            # Compute the accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    # Compute the certification accuracy
    cert_acc = 0.0
    for c in range(num_classes):
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                # Generate a random smoothing matrix
                if smoothing > 0:
                    smooth = torch.zeros_like(data).float()
                    smooth.normal_(0, smoothing)
                    data_smooth = torch.clamp(data + smooth, 0, 1)
                else:
                    data_smooth = data
                # Predict the class labels
                output = model(data_smooth)
                # Compute the robustness margin
                margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=1)[0]
                # Compute the certification accuracy
                correct_c = (margin >= -epsilon).sum().item()
                cert_acc += correct_c
    cert_acc /= num_classes * len(test_loader.dataset)
    # Print the results
    print('Test Loss: {:.6f}, Accuracy: {:.4f}%, Cert Accuracy: {:.4f}%'.format(
        test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc))
    return test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc

# Set the random seed for reproducibility
torch.manual_seed(1234)

# Define the device and hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
momentum = 0.9
weight_decay = 0.0005
epsilon = 0.05
num_iterations = 1


##batch_size = 10 # or 128 if you retrain the model with a smaller batch size
batch_size = 1
##test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# ...

'''[cert_acc = 0.0
for c in range(num_classes):
    for i in range(len(test_loader.dataset)):
        data, target = test_loader.dataset[i]
        data, target = data.unsqueeze(0).to(device), target.unsqueeze(0).to(device)
        
]'''
# Load the dataset
trainset = torchvision.datasets.ImageFolder(root='imagenette2/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testset = torchvision.datasets.ImageFolder(root='imagenette2/val', transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

radius = 0.1
# Randomly sample Gaussian noise as the smoothing noise
def sample_noise(image):
    noise = torch.randn_like(image) * radius
    return noise

num_classes = 10
# Compute the average score of the randomized model over multiple iterations
'''def randomized_score(images, labels):
    ##total_score = torch.zeros(len(images), num_classes).to(device)
    total_score = torch.zeros(images.size(0), num_classes).to(device)
    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        total_score += F.softmax(output, dim=1)
    return total_score / num_iterations
'''

def randomized_score(images, labels):
    total_score = torch.zeros(images.size(0), 1000).to(device)
    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        total_score += F.softmax(output, dim=1)
    return total_score[:, :num_classes] / num_iterations



total_loss = 0.0
total_acc = 0.0
total_cert_acc = 0.0
total_cert_radius = 0.0
total_cert_rate = 0.0
total_cert_correct = 0.0

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Compute the regular loss
        output = model(images)
        loss = criterion(output, labels)
        total_loss += loss.item() * len(images)
        _, predicted = output.max(1)
        total_acc += predicted.eq(labels).sum().item()

        # Compute the randomized smoothing loss
        randomized_scores = randomized_score(images, labels)
        randomized_predicted = randomized_scores.max(1)[1]
        cert_indices = randomized_predicted.eq(predicted)
        cert_images = images[cert_indices]
        cert_labels = labels[cert_indices]
        cert_scores = randomized_scores[cert_indices]
        cert_radius = torch.norm(sample_noise(cert_images).view(len(cert_images), -1), dim=1).mean()
        cert_correct = cert_scores.max(1)[1].eq(cert_labels).sum().item()
        cert_acc = cert_correct / len(cert_images) if len(cert_images) > 0 else 0.0
        cert_rate = len(cert_images) / len(images)

        total_cert_acc += cert_acc
        total_cert_radius += cert_radius
        total_cert_rate += cert_rate
        total_cert_correct += cert_correct

num_images = len(test_loader.dataset)
avg_loss = total_loss / num_images
avg_acc = total_acc / num_images
avg_cert_acc = total_cert_acc / num_images
avg_cert_radius = total_cert_radius / num_images
avg_cert_rate = total_cert_rate / num_images
avg_cert_correct = total_cert_correct / num_images

print('Regular test accuracy: %.2f%%' % (avg_acc * 100))
print('Certified test accuracy: %.2f%%' % (avg_cert_acc * 100))
print('Average certified radius: %.4f' % avg_cert_radius)
print('Certification rate: %.2f%%' % (avg_cert_rate * 100))
print('Average number of correct labels within certified radius: %.2f' % avg_cert_correct)


# Evaluate the model with input-dependent certification
##eval(model, device, test_loader, criterion, num_classes=10, radius=0.1, num_iterations=1, batch_size=128)
eval(model, device, test_loader, criterion, num_classes=10, radius=0.1, num_iterations=1)


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worke

RuntimeError: ignored

In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import time

# Define the dataset transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

'''# Load the training dataset
trainset = torchvision.datasets.ImageFolder(root='imagenette2', transform=transform)
##trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)'''

'''# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(7 * 7 * 512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))
        x = F.relu(self.conv7(x))
        x = self.pool(F.relu(self.conv8(x)))
        x = x.view(-1, 7 * 7 * 512)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x'''

# Load the model
model = torchvision.models.resnet18(pretrained=True)
##model.eval()

# Define the evaluation function with input-dependent certification
def eval(model, device, test_loader, epsilon, criterion, batch_size=128):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    num_classes = 10
    smoothing = int(np.ceil(epsilon * 255))
    for data, target in test_loader(batch_size=128):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            # Generate a random smoothing matrix
            if smoothing > 0:
                smooth = torch.zeros_like(data).float()
                smooth.normal_(0, smoothing)
                data_smooth = torch.clamp(data + smooth, 0, 1)
            else:
                data_smooth = data
            # Predict the class labels
            output = model(data_smooth)
            # Compute the loss
            test_loss += criterion(output, target).item() * data.size(0)
            # Compute the accuracy
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    # Compute the certification accuracy
    cert_acc = 0.0
    for c in range(num_classes):
        for data, target in test_loader(batch_size=128):
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                # Generate a random smoothing matrix
                if smoothing > 0:
                    smooth = torch.zeros_like(data).float()
                    smooth.normal_(0, smoothing)
                    data_smooth = torch.clamp(data + smooth, 0, 1)
                else:
                    data_smooth = data
                # Predict the class labels
                output = model(data_smooth)
                # Compute the robustness margin
                ##margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=1)[0]
                margin = output[:, c] - torch.max(output[:, torch.arange(num_classes) != c], axis=2)[0]

                # Compute the certification accuracy
                correct_c = (margin >= -epsilon).sum().item()
                cert_acc += correct_c
    cert_acc /= num_classes * len(test_loader.dataset)
    # Print the results
    print('Test Loss: {:.6f}, Accuracy: {:.4f}%, Cert Accuracy: {:.4f}%'.format(
        test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc))
    return test_loss / len(test_loader.dataset), 100. * correct / total, 100. * cert_acc

# Set the random seed for reproducibility
torch.manual_seed(1234)

# Define the device and hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.001
momentum = 0.9
weight_decay = 0.0005
epsilon = 0.05
num_iterations = 1


batch_size = 128 # or 128 if you retrain the model with a smaller batch size
'''input_data_batch = torch.utils.data.DataLoader(input_data, batch_size=batch_size, shuffle=True)
target_data_batch = torch.utils.data.DataLoader(target_data, batch_size=batch_size, shuffle=True)
'''
# Load the dataset
trainset = torchvision.datasets.ImageFolder(root='imagenette2/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testset = torchvision.datasets.ImageFolder(root='imagenette2/val', transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

'''# Define the neural network model, optimizer, and loss function
import torch.optim as optim
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()


    
# Define the training function
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * data.size(0)
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        train_loss /= len(train_loader.dataset)
        train_acc = 100. * correct / len(train_loader.dataset)

    return train_loss, train_acc
'''
radius = 0.1
# Randomly sample Gaussian noise as the smoothing noise
def sample_noise(image):
    noise = torch.randn_like(image) * radius
    return noise

num_classes = 10
# Compute the average score of the randomized model over multiple iterations
def randomized_score(images, labels):
    total_score = torch.zeros(len(images), num_classes).to(device)
    for i in range(num_iterations):
        noise = sample_noise(images)
        noisy_images = torch.clamp(images + noise, 0.0, 1.0)
        output = model(noisy_images)
        total_score += F.softmax(output, dim=1)
    return total_score / num_iterations

total_loss = 0.0
total_acc = 0.0
total_cert_acc = 0.0
total_cert_radius = 0.0
total_cert_rate = 0.0
total_cert_correct = 0.0

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Compute the regular loss
        output = model(images)
        loss = criterion(output, labels)
        total_loss += loss.item() * len(images)
        _, predicted = output.max(1)
        total_acc += predicted.eq(labels).sum().item()

        # Compute the randomized smoothing loss
        randomized_scores = randomized_score(images, labels)
        randomized_predicted = randomized_scores.max(1)[1]
        cert_indices = randomized_predicted.eq(predicted)
        cert_images = images[cert_indices]
        cert_labels = labels[cert_indices]
        cert_scores = randomized_scores[cert_indices]
        cert_radius = torch.norm(sample_noise(cert_images).view(len(cert_images), -1), dim=1).mean()
        cert_correct = cert_scores.max(1)[1].eq(cert_labels).sum().item()
        cert_acc = cert_correct / len(cert_images) if len(cert_images) > 0 else 0.0
        cert_rate = len(cert_images) / len(images)

        total_cert_acc += cert_acc
        total_cert_radius += cert_radius
        total_cert_rate += cert_rate
        total_cert_correct += cert_correct

num_images = len(test_loader.dataset)
avg_loss = total_loss / num_images
avg_acc = total_acc / num_images
avg_cert_acc = total_cert_acc / num_images
avg_cert_radius = total_cert_radius / num_images
avg_cert_rate = total_cert_rate / num_images
avg_cert_correct = total_cert_correct / num_images

print('Regular test accuracy: %.2f%%' % (avg_acc * 100))
print('Certified test accuracy: %.2f%%' % (avg_cert_acc * 100))
print('Average certified radius: %.4f' % avg_cert_radius)
print('Certification rate: %.2f%%' % (avg_cert_rate * 100))
print('Average number of correct labels within certified radius: %.2f' % avg_cert_correct)


'''# Train the model
train(model, device, trainloader, optimizer, criterion)'''

# Evaluate the model with input-dependent certification
eval(model, device, test_loader, criterion, num_classes=10, radius=0.1, num_iterations=1, batch_size=128)



RuntimeError: ignored

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define dataset and data loader
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Define the transformation pipeline for the input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test set
test_dir = 'imagenette2/val/'
test_dataset = ImageFolder(test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

# Load the model
model = torchvision.models.resnet18(pretrained=True)
model.eval()

'''
dataset = ImageNet(root='./imagenette2', split='train', transform=data_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train a ResNet50 model on ImageNet with Gaussian data augmentation
model = models.resnet50(pretrained=False).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

def train_model(model, optimizer, scheduler, dataloader, epochs=100, sigma=0.1):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)

            # Generate Gaussian noise
            noise = sigma * torch.randn_like(inputs).to(device)

            # Add noise to inputs
            inputs_noisy = inputs + noise

            # Compute logits
            logits = model(inputs_noisy)

            # Compute loss
            loss = torch.nn.functional.cross_entropy(logits, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            predicted = torch.argmax(logits, dim=1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

            if i % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}, Accuracy: {100*correct/total:.2f}%")

        # Update learning rate
        scheduler.step()

    # Return the trained model
    return model

# Train the model
model = train_model(model, optimizer, scheduler, dataloader, epochs=10, sigma=0.1)'''

# Define the smoothed classifier g
def smoothed_classifier(x, model, n_samples, sigma):
    model.eval()
    with torch.no_grad():
        logits = torch.zeros((n_samples,) + model(x).shape).to(device)
        for i in range(n_samples):
            noise = sigma * torch.randn_like(x).to(device)
            logits[i] = model(x + noise)
        avg_logits = torch.mean(logits, dim=0)
        return avg_logits

# Test the smoothed classifier on an example image
from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/3/32/Tom_Cruise_by_Gage_Skidmore.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = data_transform(img).unsqueeze(0).to(device)

n_samples = 100
sigma = 0.1

logits = smoothed_classifier(img, model, n_samples, sigma)
pred_label = torch.argmax(logits).item()

print(f"Predicted class: {pred_label}")


RuntimeError: ignored

In [ ]:
!curl -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar xzf imagenette2.tgz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1485M  100 1485M    0     0  42.1M      0  0:00:35  0:00:35 --:--:-- 41.3M


In [ ]:
import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

'''import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')'''

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='imagenette2/train', transform=transform)
test_dataset = ImageNetDataset(root='imagenette2/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

sigma = 0.1
n_samples = 100



# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
!git clone https://github.com/fastai/imagenette
!pip install torchvision
import tarfile

with tarfile.open('/content/imagenette/imagenette2-160.tgz', 'r:gz') as tar:
    tar.extractall('/content/named_folders/imagenet-100')


fatal: destination path 'imagenette' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


FileNotFoundError: ignored

In [ ]:
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar
!tar -xvf ILSVRC2012_img_train.tar

!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
!tar -xvf ILSVRC2012_img_val.tar

import torchvision.datasets as datasets
import os

!mkdir imagenet
# Define the path to the ImageNet data
data_path = './imagenet/'

# Define the subset of classes to use
classes = ['cat', 'dog', 'bird']

# Create a PyTorch dataset using the ImageFolder class
dataset = datasets.ImageFolder(root=data_path, 
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])
                                ]), 
                                target_transform=lambda x: classes.index(os.path.basename(os.path.dirname(x))))

# Split the dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len(dataset)-500, 500])


In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the ImageNet dataset
train_dataset = datasets.ImageNet(root='imagenet', split='train', transform=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))

# Download the dataset
train_dataset.download()


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='path/to/imagenet/train', transform=transform)
test_dataset = ImageNetDataset(root='path/to/imagenet/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

sigma = 0.1
n_samples = 100



# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
!pip install torch torchvision

In [ ]:


#!pip install torch torchvision

import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True).to(device)
model.eval()


import numpy as np
import torch.nn.functional as F

def randomized_smoothing(x, model, n_samples, noise_std):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = noise_std * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)


from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/3/32/Tom_Cruise_by_Gage_Skidmore.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = img.resize((224, 224), resample=Image.BILINEAR)

x = torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0).float().div(255).to(device)


n_samples = 100
noise_std = 0.1

y_smooth = randomized_smoothing(x, model, n_samples, noise_std)
pred_label = torch.argmax(y_smooth).item()

print(f"Predicted class: {pred_label}")


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import torch.nn.functional as F

# Define the standard deviation of the Gaussian noise
sigma = 0.1

# Define the number of samples for randomized smoothing
n_samples = 50

# Define the batch size for evaluation
batch_size = 64

# Define the device to run the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the ImageNet dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
trainset = torchvision.datasets.ImageFolder(root='./imagenet/train', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# Train a ResNet50 model on the dataset
model = models.resnet50(pretrained=False).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
for epoch in range(10):
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Define the randomized smoothing function
def smoothed_classifier(x, model, n_samples, sigma):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = sigma * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)

# Evaluate the smoothed classifier on the test set
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
testset = torchvision.datasets.ImageFolder(root='./imagenet/val', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

total, correct = 0, 0
for i, (inputs, labels) in enumerate(testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = smoothed_classifier(inputs, model, n_samples, sigma)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print('Accuracy of the smoothed classifier on the test set: %.2f%%' % accuracy)
